In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [2]:
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/base de dados/dataset_balanceado_limpo.csv', encoding='utf-8', sep=',')

Mounted at /content/drive


In [3]:
def normalize_dataframe(df):
    for coluna in df.columns:
        if df[coluna].dtype != 'category':  # Verifica se a coluna é numérica
            max_valor = df[coluna].max()
            df[coluna] = df[coluna] / max_valor  # Divide todos os valores da coluna pelo máximo

In [4]:
# Separe os rótulos do conjunto de dados
labels = df[' Label']  # Substitua 'rotulo' pelo nome da coluna de rótulos

# Remova a coluna de rótulos do conjunto de dados
data = df.drop(columns=[' Label'])  # Substitua 'rotulo' pelo nome da coluna de rótulos]

label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [5]:
data = data.drop(columns=['Bwd Avg Bulk Rate', ' Flow Packets/s', ' Bwd Avg Packets/Bulk', ' Bwd Avg Bytes/Bulk', ' Fwd Avg Bulk Rate', ' Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', ' ECE Flag Count', ' PSH Flag Count', 'FIN Flag Count', ' Bwd URG Flags', ' Fwd URG Flags', ' Bwd PSH Flags'], axis=1)

In [6]:
indices_selecionados = [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 32, 34, 35, 36, 37, 38, 41, 43, 44, 47, 55, 56, 58, 66, 67]
data = data.iloc[:, indices_selecionados]

In [7]:
normalize_dataframe(data)

In [ ]:
# Defina a dimensionalidade dos dados
data_dim = data.shape[1]

# Modelo Discriminador
discriminator = keras.Sequential([
    layers.Reshape((35, 1), input_shape=(35,)),  # Camada de entrada com forma (35, 1)
    layers.Conv1D(128, 16, activation='relu'),
    layers.Conv1D(64, 8, activation='softmax'),# Primeira camada de convolução
    layers.MaxPooling1D(2),  # Camada de max pooling
    layers.Flatten(),  # Camada de achatamento
    layers.Dense(1024, activation='relu'),
    layers.Dense(1024, activation='relu'),# Camada densa intermediária
    layers.Dense(7, activation='softmax')  # Camada de saída com ativação Softmax
  ])

discriminator.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Modelo Gerador
latent_dim = 100
generator = keras.Sequential([
    layers.Input(shape=(latent_dim + 7,)),  # O espaço latente mais a condição de classe
    layers.Dense(1024, activation='relu'),
    layers.Dense(1024, activation='relu'),
    layers.Dense(1024, activation='relu'),
    layers.Dense(1024, activation='relu'),
    layers.Dense(data_dim, activation='tanh')  # A camada de saída gera os dados
])

# Modelo GAN
discriminator.trainable = True  # Congela o discriminador durante o treinamento da GAN
gan_input = keras.Input(shape=(latent_dim + 7,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = keras.models.Model(gan_input, gan_output)

gan.compile(loss='categorical_crossentropy', optimizer='adam')

# Hiperparâmetros do treinamento
batch_size = 100
epochs = 5

# Treinamento da GAN condicionada
for e in range(epochs):
    for _ in range(len(data) // batch_size):
        # Amostras aleatórias do espaço latente e condição de classe
        batch_indices = np.random.choice(len(data), batch_size, replace=False)
        batch_data = data.iloc[batch_indices]
        batch_labels = labels_encoded[batch_indices]

        noise = np.random.randn(batch_size, latent_dim)
        class_condition = np.random.randint(0, 7, batch_size)
        class_one_hot = keras.utils.to_categorical(class_condition, num_classes=7)
        gan_input = np.concatenate([noise, class_one_hot], axis=1)

        generated_data = generator.predict(gan_input)

        # Crie um vetor de rótulos para o discriminador
        # Um vetor de zeros para dados gerados e um vetor de uns para dados reais
        y_dis_generated = np.zeros((batch_size, 7))
        y_dis_real = keras.utils.to_categorical(batch_labels, num_classes=7)
        y_dis = np.vstack([y_dis_real, y_dis_generated])

        X = np.vstack([batch_data, generated_data])

        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(X, y_dis)

        # Agora, gere amostras condicionadas para treinar o gerador
        noise = np.random.randn(batch_size, latent_dim)
        class_condition = np.random.randint(0, 7, batch_size)
        class_one_hot = keras.utils.to_categorical(class_condition, num_classes=7)
        gan_input = np.concatenate([noise, class_one_hot], axis=1)

        # Crie um vetor de rótulos para o treinamento do gerador
        # Aqui, você deseja que o gerador pense que as amostras são reais, então os rótulos são todos 1
        y_gen = np.ones((batch_size, 7))

        discriminator.trainable = True
        g_loss = gan.train_on_batch(gan_input, y_gen)

    #if e % 1000 == 0:
        print(f"Época: {e}, Divergência Discriminador: {d_loss[0]}, Acurácia Discriminador: {100 * d_loss[1]:.2f}%, Divergência GAN: {g_loss}")



In [ ]:
drive.mount('/content/drive')

df_teste = pd.read_csv('/content/drive/MyDrive/base de dados/dataset_teste.csv', encoding='utf-8', sep=',')

In [ ]:
labels_teste = df_teste[' Label']

data_teste = df_teste.drop(columns=[' Label'])

label_encoder = LabelEncoder()
labels_encoded_teste = label_encoder.fit_transform(labels_teste)
y_teste = to_categorical(labels_encoded_teste, num_classes=7)

In [ ]:
data_teste = data_teste.drop(columns=['Bwd Avg Bulk Rate', ' Flow Packets/s', ' Bwd Avg Packets/Bulk', ' Bwd Avg Bytes/Bulk', ' Fwd Avg Bulk Rate', ' Fwd Avg Packets/Bulk', 'Fwd Avg Bytes/Bulk', ' ECE Flag Count', ' PSH Flag Count', 'FIN Flag Count', ' Bwd URG Flags', ' Fwd URG Flags', ' Bwd PSH Flags'], axis=1)
indices_selecionados = [0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 32, 34, 35, 36, 37, 38, 41, 43, 44, 47, 55, 56, 58, 66, 67]
data_teste = data_teste.iloc[:, indices_selecionados]

In [ ]:
normalize_dataframe(data_teste)

In [ ]:
loss, accuracy = discriminator.evaluate(data_teste, y_teste, verbose=1, steps=2000)

print('Acuracia:', accuracy)
print('loss', loss)